In [1]:
from llama_index.core.selectors import LLMSingleSelector
import nest_asyncio
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
import torch
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage
from datasets import load_dataset
from llama_index.llms.openai import OpenAI
from groq import Groq
from openai import OpenAI

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
OPENAI_API_KEY = "INSERT_OPEN_API_KEY_HERE"
GROQ_API_KEY = "INSERT_GROQ_API_KEY_HERE"

In [8]:
client = OpenAI(api_key="INSERT_OPEN_API_KEY_HERE")

def prompt_gpt(messages):
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
  )
  return completion.choices[0].message.content

In [9]:
def prompt_llama(messages):
  client = Groq(api_key=GROQ_API_KEY)

  chat_completion = client.chat.completions.create(
      messages=messages,
      model="llama3-8b-8192",
  )

  return chat_completion.choices[0].message.content

In [ ]:
dataset_validation = load_dataset("openlifescienceai/mmlu_anatomy", split="test")

In [ ]:
print(dataset_validation)

# evaluating llama gpt

In [51]:
import time
from tqdm import tqdm

def predict_llama_gpt(prompt, options):
  time.sleep(2.2)

  llama_prompt = [{"role": "user", "content": prompt}]
  gpt_prompt = [{"role": "user", "content": prompt}]

  llama_explanation = prompt_llama(llama_prompt)
  gpt_explanation = prompt_gpt(gpt_prompt)

  llama_prompt.append({"role": "assistant", "content": llama_explanation})
  llama_prompt.append({"role": "user", "content": "Here is an alternative reasoning that may or may not be correct. Decide whether you prefer this reasoning or your own, then explain again: \n" + gpt_explanation})

  llama_reconsideration = prompt_llama(llama_prompt)

  llama_prompt.append({"role": "assistant", "content": llama_reconsideration})
  llama_prompt.append({"role": "user", "content": f'''Now I need the answer in exact form of one of the options. Here are the four options again
{options[0]}
{options[1]}
{options[2]}
{options[3]}

Based on your explanation, respond with the correct choice from the list above verbatim. Do not include any other text in your response.
'''})

  llama_prediction = prompt_llama(llama_prompt)

  return llama_prediction


def evaluate_llama_gpt():
  SAMPLE_CNT = len(dataset_validation)
  mismatch_cnt = 0
  predictions = []
  references = []
  right_indexes = []
  wrong_indexes = []

  for i in tqdm(range(SAMPLE_CNT)):
    example = dataset_validation[i]["data"]
    question, option_a, option_b, option_c, option_d = example["Question"], example['Options']['A'], example['Options']['B'], example['Options']['C'], example['Options']['D']
    prompt = f'''{question}

{option_a}
{option_b}
{option_c}
{option_d}

Respond with the correct choice from the list above and explain your answer.'''
# Respond with the correct choice from the list above verbatim. Do not include any other text in your response.'''

    options = [example["Options"]["A"], example["Options"]["B"], example["Options"]["C"], example["Options"]["D"]]
    correct_option = example["Correct Answer"]
    references.append(correct_option)

    prediction = predict_llama_gpt(prompt, options)

    predictions.append(prediction)

    mismatch_cnt += prediction not in options

    if prediction != correct_option:
      wrong_indexes.append(i)
    else:
      right_indexes.append(i)

  exact_match = sum([prediction == reference for prediction, reference, in zip(predictions, references)]) / SAMPLE_CNT
  mismatch = mismatch_cnt / SAMPLE_CNT

  print(right_indexes, '\n', wrong_indexes)
  return exact_match, mismatch

exact_match, mismatch = evaluate_llama_gpt()

  0%|          | 0/135 [00:00<?, ?it/s]

100%|██████████| 135/135 [18:34<00:00,  8.25s/it]

[0, 1, 2, 4, 6, 7, 10, 16, 17, 18, 19, 20, 26, 27, 28, 30, 31, 32, 36, 37, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 56, 57, 58, 60, 62, 63, 65, 66, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 89, 90, 92, 93, 96, 98, 100, 101, 102, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 120, 122, 123, 124, 126, 127, 128, 130, 131, 132, 133] 
 [3, 5, 8, 9, 11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 29, 33, 34, 35, 38, 41, 52, 54, 55, 59, 61, 64, 68, 70, 85, 88, 91, 94, 95, 97, 99, 103, 104, 105, 107, 110, 119, 121, 125, 129, 134]


In [52]:
print(exact_match)
print(mismatch)

0.6666666666666666
0.044444444444444446


# evaluating gpt llama

In [53]:
import time
from tqdm import tqdm

def predict_gpt_llama(prompt, options):
  time.sleep(2.2)

  llama_prompt = [{"role": "user", "content": prompt}]
  gpt_prompt = [{"role": "user", "content": prompt}]

  llama_explanation = prompt_llama(llama_prompt)
  gpt_explanation = prompt_gpt(gpt_prompt)

  gpt_prompt.append({"role": "assistant", "content": gpt_explanation})
  gpt_prompt.append({"role": "user", "content": "Here is an alternative reasoning that may or may not be correct. Decide whether you prefer this reasoning or your own, then explain again: \n" + llama_explanation})

  gpt_reconsideration = prompt_gpt(gpt_prompt)

  gpt_prompt.append({"role": "assistant", "content": gpt_reconsideration})
  gpt_prompt.append({"role": "user", "content": f'''Now I need the answer in exact form of one of the options. Here are the four options again
{options[0]}
{options[1]}
{options[2]}
{options[3]}

Based on your explanation, respond with the correct choice from the list above verbatim. Do not include any other text in your response.
'''})

  gpt_prediction = prompt_gpt(gpt_prompt)

  return gpt_prediction


def evaluate_gpt_llama():
  SAMPLE_CNT = len(dataset_validation)
  mismatch_cnt = 0
  predictions = []
  references = []
  right_indexes = []
  wrong_indexes = []

  for i in tqdm(range(SAMPLE_CNT)):
    example = dataset_validation[i]["data"]
    question, option_a, option_b, option_c, option_d = example["Question"], example['Options']['A'], example['Options']['B'], example['Options']['C'], example['Options']['D']
    prompt = f'''{question}

{option_a}
{option_b}
{option_c}
{option_d}

Respond with the correct choice from the list above and explain your answer.'''
# Respond with the correct choice from the list above verbatim. Do not include any other text in your response.'''

    options = [example["Options"]["A"], example["Options"]["B"], example["Options"]["C"], example["Options"]["D"]]
    correct_option = example["Correct Answer"]
    references.append(correct_option)

    prediction = predict_gpt_llama(prompt, options)

    predictions.append(prediction)

    mismatch_cnt += prediction not in options

    if prediction != correct_option:
      wrong_indexes.append(i)
    else:
      right_indexes.append(i)

  exact_match = sum([prediction == reference for prediction, reference, in zip(predictions, references)]) / SAMPLE_CNT
  mismatch = mismatch_cnt / SAMPLE_CNT

  print(right_indexes, '\n', wrong_indexes)
  return exact_match, mismatch

exact_match, mismatch = evaluate_gpt_llama()

100%|██████████| 135/135 [18:14<00:00,  8.11s/it]

[0, 2, 4, 6, 7, 12, 14, 15, 16, 17, 19, 20, 26, 27, 28, 30, 32, 36, 37, 39, 40, 42, 44, 45, 46, 47, 48, 49, 50, 51, 57, 58, 60, 61, 62, 63, 64, 65, 66, 71, 72, 73, 76, 77, 78, 79, 80, 82, 83, 84, 89, 90, 93, 98, 100, 102, 106, 108, 111, 112, 113, 114, 115, 117, 118, 120, 122, 123, 124, 126, 127, 128, 130, 131, 133, 134] 
 [1, 3, 5, 8, 9, 10, 11, 13, 18, 21, 22, 23, 24, 25, 29, 31, 33, 34, 35, 38, 41, 43, 52, 53, 54, 55, 56, 59, 67, 68, 69, 70, 74, 75, 81, 85, 86, 87, 88, 91, 92, 94, 95, 96, 97, 99, 101, 103, 104, 105, 107, 109, 110, 116, 119, 121, 125, 129, 132]


In [54]:
print(exact_match)
print(mismatch)

0.562962962962963
0.06666666666666667


# evaluating llama alone

In [55]:
import time
from tqdm import tqdm

def predict_llama(prompt):
  time.sleep(2.2)

  llama_prompt = [{"role": "user", "content": prompt}]
  return prompt_llama(llama_prompt)

def evaluate_llama():
  SAMPLE_CNT = len(dataset_validation)
  mismatch_cnt = 0
  predictions = []
  references = []
  right_indexes = []
  wrong_indexes = []

  for i in tqdm(range(SAMPLE_CNT)):
    example = dataset_validation[i]["data"]
    question, option_a, option_b, option_c, option_d = example["Question"], example['Options']['A'], example['Options']['B'], example['Options']['C'], example['Options']['D']
    prompt = f'''{question}

{option_a}
{option_b}
{option_c}
{option_d}

Respond with the correct choice from the list above verbatim. Do not include any other text in your response.'''
# Respond with the correct choice from the list above verbatim. Do not include any other text in your response.'''

    options = [example["Options"]["A"], example["Options"]["B"], example["Options"]["C"], example["Options"]["D"]]
    correct_option = example["Correct Answer"]
    references.append(correct_option)

    prediction = predict_llama(prompt)

    predictions.append(prediction)

    mismatch_cnt += prediction not in options

    if prediction != correct_option:
      wrong_indexes.append(i)
    else:
      right_indexes.append(i)

  exact_match = sum([prediction == reference for prediction, reference, in zip(predictions, references)]) / SAMPLE_CNT
  mismatch = mismatch_cnt / SAMPLE_CNT
  
  print(right_indexes, '\n', wrong_indexes)

  return exact_match, mismatch

exact_match, mismatch = evaluate_llama()

100%|██████████| 135/135 [06:16<00:00,  2.79s/it]

[0, 2, 4, 6, 7, 9, 15, 17, 18, 19, 20, 26, 27, 28, 30, 32, 36, 37, 39, 40, 42, 45, 46, 47, 50, 51, 53, 54, 57, 58, 60, 61, 62, 63, 64, 65, 66, 70, 71, 72, 75, 76, 78, 79, 80, 81, 82, 83, 85, 89, 90, 93, 96, 98, 100, 102, 106, 108, 110, 111, 112, 113, 114, 115, 117, 118, 120, 121, 122, 123, 126, 127, 128, 130, 131, 133, 134] 
 [1, 3, 5, 8, 10, 11, 12, 13, 14, 16, 21, 22, 23, 24, 25, 29, 31, 33, 34, 35, 38, 41, 43, 44, 48, 49, 52, 55, 56, 59, 67, 68, 69, 73, 74, 77, 84, 86, 87, 88, 91, 92, 94, 95, 97, 99, 101, 103, 104, 105, 107, 109, 116, 119, 124, 125, 129, 132]


In [56]:
print(exact_match)
print(mismatch)

0.5703703703703704
0.11851851851851852


# gpt alone

In [57]:
import time
from tqdm import tqdm

def predict_gpt(prompt):
  time.sleep(2.2)

  prompt = [{"role": "user", "content": prompt}]
  return prompt_gpt(prompt)

def evaluate_gpt():
  SAMPLE_CNT = len(dataset_validation)
  mismatch_cnt = 0
  predictions = []
  references = []
  right_indexes = []
  wrong_indexes = []

  for i in tqdm(range(SAMPLE_CNT)):
    example = dataset_validation[i]["data"]
    question, option_a, option_b, option_c, option_d = example["Question"], example['Options']['A'], example['Options']['B'], example['Options']['C'], example['Options']['D']
    prompt = f'''{question}

{option_a}
{option_b}
{option_c}
{option_d}

Respond with the correct choice from the list above verbatim. Do not include any other text in your response.'''
# Respond with the correct choice from the list above verbatim. Do not include any other text in your response.'''

    options = [example["Options"]["A"], example["Options"]["B"], example["Options"]["C"], example["Options"]["D"]]
    correct_option = example["Correct Answer"]
    references.append(correct_option)

    prediction = predict_gpt(prompt)

    predictions.append(prediction)

    mismatch_cnt += prediction not in options

    if prediction != correct_option:
      wrong_indexes.append(i)
    else:
      right_indexes.append(i)

  print(right_indexes, '\n', wrong_indexes)

  exact_match = sum([prediction == reference for prediction, reference, in zip(predictions, references)]) / SAMPLE_CNT
  mismatch = mismatch_cnt / SAMPLE_CNT

  return exact_match, mismatch

exact_match, mismatch = evaluate_gpt()

100%|██████████| 135/135 [06:13<00:00,  2.77s/it]

[0, 1, 2, 3, 4, 6, 7, 8, 10, 17, 18, 19, 20, 21, 26, 27, 28, 30, 31, 32, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 56, 57, 58, 60, 62, 63, 64, 66, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 83, 84, 86, 89, 90, 96, 98, 100, 101, 102, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 119, 120, 122, 123, 124, 126, 128, 129, 130, 131, 132, 133] 
 [5, 9, 11, 12, 13, 14, 15, 16, 22, 23, 24, 25, 29, 33, 34, 35, 38, 52, 54, 55, 59, 61, 65, 67, 68, 79, 81, 82, 85, 87, 88, 91, 92, 93, 94, 95, 97, 99, 103, 104, 105, 107, 110, 118, 121, 125, 127, 134]


In [58]:
print(exact_match)
print(mismatch)

0.6444444444444445
0.037037037037037035


# stat processing

In [ ]:
llama_right_indexes_medmcqa_anatomy = [1, 2, 5, 6, 7, 9, 10, 13, 14, 17, 19, 21, 22, 25, 27, 29, 30, 34, 35, 37, 40, 41, 43, 44, 48, 51, 53, 54, 57, 58, 59, 60, 61, 64, 68, 69, 72, 74, 77, 79, 80, 81, 82, 84, 85, 87, 88, 90, 91, 94, 97, 98, 99, 102, 106, 107, 108, 109, 112, 113, 114, 116, 118, 119, 120, 123, 126, 129, 130, 132, 134, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 153, 155, 157, 160, 163, 164, 165, 166, 168, 170, 171, 172, 173, 174, 175, 177, 178, 179, 182, 184, 185, 192, 194, 195, 197, 200, 201, 204, 205, 213, 215, 217, 219, 220, 222, 223, 224, 227, 228, 230, 232, 233] 

llama_wrong_indexes_medmcqa_anatomy = [0, 3, 4, 8, 11, 12, 15, 16, 18, 20, 23, 24, 26, 28, 31, 32, 33, 36, 38, 39, 42, 45, 46, 47, 49, 50, 52, 55, 56, 62, 63, 65, 66, 67, 70, 71, 73, 75, 76, 78, 83, 86, 89, 92, 93, 95, 96, 100, 101, 103, 104, 105, 110, 111, 115, 117, 121, 122, 124, 125, 127, 128, 131, 133, 135, 144, 151, 152, 154, 156, 158, 159, 161, 162, 167, 169, 176, 180, 181, 183, 186, 187, 188, 189, 190, 191, 193, 196, 198, 199, 202, 203, 206, 207, 208, 209, 210, 211, 212, 214, 216, 218, 221, 225, 226, 229, 231]

gpt_right_indexes_medmcqa_anatomy = [1, 4, 5, 6, 7, 9, 14, 17, 18, 20, 21, 22, 23, 24, 25, 27, 29, 30, 35, 37, 40, 44, 53, 54, 56, 57, 60, 61, 64, 68, 69, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 89, 90, 91, 95, 97, 98, 100, 101, 103, 105, 107, 108, 111, 112, 113, 114, 119, 121, 122, 123, 125, 129, 131, 134, 135, 138, 139, 140, 142, 145, 146, 147, 148, 150, 153, 155, 157, 158, 159, 161, 162, 163, 164, 165, 166, 170, 171, 172, 174, 175, 176, 177, 178, 179, 181, 182, 184, 187, 190, 192, 193, 194, 195, 202, 205, 206, 209, 210, 213, 214, 215, 216, 219, 220, 222, 223, 224, 225, 227, 228, 229, 231, 232] 

gpt_wrong_indexes_medmcqa_anatomy = [0, 2, 3, 8, 10, 11, 12, 13, 15, 16, 19, 26, 28, 31, 32, 33, 34, 36, 38, 39, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 55, 58, 59, 62, 63, 65, 66, 67, 70, 71, 83, 88, 92, 93, 94, 96, 99, 102, 104, 106, 109, 110, 115, 116, 117, 118, 120, 124, 126, 127, 128, 130, 132, 133, 136, 137, 141, 143, 144, 149, 151, 152, 154, 156, 160, 167, 168, 169, 173, 180, 183, 185, 186, 188, 189, 191, 196, 197, 198, 199, 200, 201, 203, 204, 207, 208, 211, 212, 217, 218, 221, 226, 230, 233]

In [ ]:
llama_right_indexes_medmcqa_mixed = [0, 1, 2, 3, 4, 7, 10, 12, 13, 15, 16, 17, 21, 22, 26, 30, 31, 32, 34, 35, 42, 49, 50, 51, 53, 55, 62, 63, 64, 65, 68, 70, 71, 72, 73, 75, 76, 79, 80, 81, 82, 83, 85, 86, 88, 91, 94, 95, 98, 100, 101, 102, 103, 105, 107, 114, 115, 116, 117, 120, 121, 124, 125, 126, 127, 128, 130, 132, 134, 139, 141, 142, 146, 151, 154, 156, 157, 159, 164, 168, 171, 173, 177, 178, 182, 184, 188, 190, 193, 196, 203, 204, 209, 210, 212, 213, 216, 217, 218, 220, 221, 222, 223, 224, 232] 

llama_wrong_indexes_medmcqa_mixed = [5, 6, 8, 9, 11, 14, 18, 19, 20, 23, 24, 25, 27, 28, 29, 33, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 52, 54, 56, 57, 58, 59, 60, 61, 66, 67, 69, 74, 77, 78, 84, 87, 89, 90, 92, 93, 96, 97, 99, 104, 106, 108, 109, 110, 111, 112, 113, 118, 119, 122, 123, 129, 131, 133, 135, 136, 137, 138, 140, 143, 144, 145, 147, 148, 149, 150, 152, 153, 155, 158, 160, 161, 162, 163, 165, 166, 167, 169, 170, 172, 174, 175, 176, 179, 180, 181, 183, 185, 186, 187, 189, 191, 192, 194, 195, 197, 198, 199, 200, 201, 202, 205, 206, 207, 208, 211, 214, 215, 219, 225, 226, 227, 228, 229, 230, 231, 233]


gpt_right_indexes_medmcqa_mixed = [0, 1, 2, 3, 4, 5, 6, 12, 13, 14, 15, 16, 17, 20, 22, 25, 30, 31, 32, 34, 36, 39, 42, 43, 47, 50, 51, 52, 55, 57, 59, 61, 63, 64, 68, 70, 71, 72, 73, 74, 79, 81, 83, 85, 86, 88, 90, 91, 94, 95, 96, 98, 100, 101, 102, 103, 105, 107, 114, 115, 117, 120, 121, 124, 125, 128, 130, 131, 132, 133, 138, 139, 140, 142, 145, 146, 147, 151, 153, 154, 155, 157, 159, 163, 164, 165, 173, 174, 175, 177, 178, 181, 184, 186, 187, 188, 189, 190, 196, 199, 202, 203, 204, 206, 211, 212, 214, 216, 217, 219, 220, 222, 223, 224, 228, 230, 232] 

gpt_wrong_indexes_medmcqa_mixed = [7, 8, 9, 10, 11, 18, 19, 21, 23, 24, 26, 27, 28, 29, 33, 35, 37, 38, 40, 41, 44, 45, 46, 48, 49, 53, 54, 56, 58, 60, 62, 65, 66, 67, 69, 75, 76, 77, 78, 80, 82, 84, 87, 89, 92, 93, 97, 99, 104, 106, 108, 109, 110, 111, 112, 113, 116, 118, 119, 122, 123, 126, 127, 129, 134, 135, 136, 137, 141, 143, 144, 148, 149, 150, 152, 156, 158, 160, 161, 162, 166, 167, 168, 169, 170, 171, 172, 176, 179, 180, 182, 183, 185, 191, 192, 193, 194, 195, 197, 198, 200, 201, 205, 207, 208, 209, 210, 213, 215, 218, 221, 225, 226, 227, 229, 231, 233]

In [ ]:
llama_right_indexes_medqa = [2, 4, 7, 9, 10, 11, 12, 14, 16, 18, 21, 22, 24, 25, 27, 28, 30, 33, 34, 35, 37, 38, 40, 41, 45, 47, 48, 49, 51, 54, 55, 57, 61, 62, 63, 64, 66, 67, 70, 71, 76, 77, 79, 80, 83, 84, 85, 86, 91, 92, 97, 98, 101, 102, 103, 104, 105, 107, 110, 111, 114, 117, 118, 119, 120, 121, 122, 123, 125, 126, 127, 129, 132, 133, 136, 138, 140, 142, 143, 144, 147, 148, 151, 153, 154, 156, 157, 158, 160, 161, 162, 164, 167, 168, 169, 175, 177, 179, 185, 187, 188, 191, 192, 193, 194, 195, 196, 197, 199, 205, 206, 207, 208, 209, 211, 213, 218, 223, 224, 227, 229, 230, 231] 

llama_wrong_indexes_medqa = [0, 1, 3, 5, 6, 8, 13, 15, 17, 19, 20, 23, 26, 29, 31, 32, 36, 39, 42, 43, 44, 46, 50, 52, 53, 56, 58, 59, 60, 65, 68, 69, 72, 73, 74, 75, 78, 81, 82, 87, 88, 89, 90, 93, 94, 95, 96, 99, 100, 106, 108, 109, 112, 113, 115, 116, 124, 128, 130, 131, 134, 135, 137, 139, 141, 145, 146, 149, 150, 152, 155, 159, 163, 165, 166, 170, 171, 172, 173, 174, 176, 178, 180, 181, 182, 183, 184, 186, 189, 190, 198, 200, 201, 202, 203, 204, 210, 212, 214, 215, 216, 217, 219, 220, 221, 222, 225, 226, 228, 232, 233]

gpt_right_indexes_medqa = [0, 2, 4, 5, 7, 10, 11, 12, 14, 16, 17, 18, 20, 21, 22, 24, 26, 27, 28, 30, 32, 34, 35, 37, 38, 39, 40, 41, 42, 45, 49, 50, 51, 53, 54, 55, 56, 57, 58, 60, 61, 63, 64, 65, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 97, 98, 99, 101, 102, 103, 104, 105, 107, 108, 110, 111, 113, 115, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 129, 131, 132, 133, 134, 136, 137, 138, 140, 142, 143, 144, 146, 147, 148, 150, 153, 154, 155, 156, 157, 158, 161, 162, 163, 164, 166, 167, 168, 169, 172, 174, 176, 177, 179, 182, 187, 189, 192, 193, 194, 195, 196, 197, 200, 206, 207, 209, 213, 217, 218, 219, 223, 224, 227, 229, 230, 233] 

gpt_wrong_indexes_medqa = [1, 3, 6, 8, 9, 13, 15, 19, 23, 25, 29, 31, 33, 36, 43, 44, 46, 47, 48, 52, 59, 62, 66, 68, 78, 81, 93, 94, 96, 100, 106, 109, 112, 114, 116, 125, 130, 135, 139, 141, 145, 149, 151, 152, 159, 160, 165, 170, 171, 173, 175, 178, 180, 181, 183, 184, 185, 186, 188, 190, 191, 198, 199, 201, 202, 203, 204, 205, 208, 210, 211, 212, 214, 215, 216, 220, 221, 222, 225, 226, 228, 231, 232]
